Author: <b>Deep Joshi</b>
Model Name: 1D Convulation Network

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
train_df = pd.read_csv('data/UNSW_NB15_training-set.csv')
test_df = pd.read_csv('data/UNSW_NB15_testing-set.csv')

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175341 entries, 0 to 175340
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 175341 non-null  int64  
 1   dur                175341 non-null  float64
 2   proto              175341 non-null  object 
 3   service            175341 non-null  object 
 4   state              175341 non-null  object 
 5   spkts              175341 non-null  int64  
 6   dpkts              175341 non-null  int64  
 7   sbytes             175341 non-null  int64  
 8   dbytes             175341 non-null  int64  
 9   rate               175341 non-null  float64
 10  sttl               175341 non-null  int64  
 11  dttl               175341 non-null  int64  
 12  sload              175341 non-null  float64
 13  dload              175341 non-null  float64
 14  sloss              175341 non-null  int64  
 15  dloss              175341 non-null  int64  
 16  si

In [4]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 82332 non-null  int64  
 1   dur                82332 non-null  float64
 2   proto              82332 non-null  object 
 3   service            82332 non-null  object 
 4   state              82332 non-null  object 
 5   spkts              82332 non-null  int64  
 6   dpkts              82332 non-null  int64  
 7   sbytes             82332 non-null  int64  
 8   dbytes             82332 non-null  int64  
 9   rate               82332 non-null  float64
 10  sttl               82332 non-null  int64  
 11  dttl               82332 non-null  int64  
 12  sload              82332 non-null  float64
 13  dload              82332 non-null  float64
 14  sloss              82332 non-null  int64  
 15  dloss              82332 non-null  int64  
 16  sinpkt             823

In [5]:
train_df['attack_cat'] = train_df['attack_cat'].map(
    {
  "Analysis": 0,
  "Backdoor": 1,
  "DoS": 2,
  "Exploits": 3,
  "Fuzzers": 4,
  "Generic": 5,
  "Normal": 6,
  "Reconnaissance": 7,
  "Shellcode": 8,
  "Worms": 9
}
)

In [6]:
test_df['attack_cat'] = test_df['attack_cat'].map(
    {
  "Analysis": 0,
  "Backdoor": 1,
  "DoS": 2,
  "Exploits": 3,
  "Fuzzers": 4,
  "Generic": 5,
  "Normal": 6,
  "Reconnaissance": 7,
  "Shellcode": 8,
  "Worms": 9
}
)

In [7]:
# y_train = train_df['label'].values
# X_train = train_df.drop('label', axis=1).values
#
# y_test = test_df['label'].values
# X_test = test_df.drop('label', axis=1).values

X_train = train_df[['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sload', 'dload', 'smean', 'dmean', 'attack_cat']].values
y_train = train_df['label'].values

X_test = test_df[['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sload', 'dload', 'smean', 'dmean', 'attack_cat']].values
y_test = test_df['label'].values

In [8]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [9]:
BATCH_SIZE = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Data loaded from CSV and prepared for PyTorch.")
print("Training data shape:", X_train_tensor.shape)
print("Test data shape:", X_test_tensor.shape)

Data loaded from CSV and prepared for PyTorch.
Training data shape: torch.Size([175341, 1, 11])
Test data shape: torch.Size([82332, 1, 11])


In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [37]:
class Simple1DCNN(nn.Module):
    def __init__(self, num_classes=2, SEQ_LENGTH=None):
        super(Simple1DCNN, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)



        final_seq_length = SEQ_LENGTH // 4


        if final_seq_length < 1:
            raise ValueError(
                f"Calculated final_seq_length ({final_seq_length}) is too small. "
                f"With SEQ_LENGTH={SEQ_LENGTH}, it must be at least 4 for this CNN architecture."
            )

        self.fc = nn.Linear(32 * final_seq_length, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))

        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

INPUT_FEATURES_COUNT = 11

model = Simple1DCNN(SEQ_LENGTH=INPUT_FEATURES_COUNT)
model.to(device)
print(model)

Simple1DCNN(
  (conv1): Conv1d(1, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu1): ReLU()
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu2): ReLU()
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)


In [40]:

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        # loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Add this line
        optimizer.step()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * 16 + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

In [41]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
batch_size = 16
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# epochs = 5
epochs = 2

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1901094393348096.000000  [   16/175341]
loss: 0.686958  [ 1616/175341]
loss: 0.712080  [ 3216/175341]
loss: 0.710116  [ 4816/175341]
loss: 0.697722  [ 6416/175341]
loss: 0.694433  [ 8016/175341]
loss: 0.690139  [ 9616/175341]
loss: 0.683523  [11216/175341]
loss: 0.687556  [12816/175341]
loss: 0.659308  [14416/175341]
loss: 0.675219  [16016/175341]
loss: 0.698163  [17616/175341]
loss: 0.656731  [19216/175341]
loss: 0.638151  [20816/175341]
loss: 0.616400  [22416/175341]
loss: 0.685584  [24016/175341]
loss: 0.666013  [25616/175341]
loss: 0.665342  [27216/175341]
loss: 0.642537  [28816/175341]
loss: 0.571696  [30416/175341]
loss: 0.687527  [32016/175341]
loss: 0.663039  [33616/175341]
loss: 0.688568  [35216/175341]
loss: 0.608793  [36816/175341]
loss: 0.773537  [38416/175341]
loss: 0.633445  [40016/175341]
loss: 0.632659  [41616/175341]
loss: 0.691394  [43216/175341]
loss: 0.631383  [44816/175341]
loss: 0.630635  [46416/175341]
loss: 0.503534 